### Installing and Importing Necessary Packages

In [1]:
!pip install chatterbot

     |████████████████████████████████| 71kB 3.3MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import nltk
import spacy
import re
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement
import random

import warnings
warnings.filterwarnings(action='ignore')

!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


# Chatbot Assignment

In this assignment, you'll work with a dataset called Cornell Movie--Dialogs Corpus, which was released by the Cornell University. The dataset contains conversations from more than 600 movies.



In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

dialogs_df = pd.read_sql_query('select * from dialogs',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [4]:
dialogs_df

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...
...,...,...
304441,304441,Lord Chelmsford seems to want me to stay back ...
304442,304442,I think Chelmsford wants a good man on the bor...
304443,304443,"Well I assure you, Sir, I have no desire to cr..."
304444,304444,"And I assure you, you do not In fact I'd be ob..."


### 1. First, do some data preprocessing to clean up the data. You can use your solution to the assignment of the Text preprocessing checkpoint.

In [5]:
nlp = spacy.load('en',disable=['parser','ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done here. 
dialogs_doc = nlp(" ".join(dialogs_df.dialogs))

In [6]:
dialogs_sents = [sent.text for sent in dialogs_doc.sents if len(sent.text) > 1] # Only include sentences with more than one word. 

### 2. Develop a chatbot using this corpus. In doing this, you're free to choose a chatbot development library like ChatterBot or write your own code from scratch.

In [7]:
GREETING_INPUTS = ['hello','hi','greetings',"what's up", "hey"]
GREETING_RESPONSES = ['hello','hi','hey','hi there']
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
      return random.choice(GREETING_RESPONSES)

In [8]:
# creating a chatbot with ChatterBot! 
chatbot = ChatBot('Dialogs')

# this is to remove the accumulated knowledge base (by why?)
chatbot.storage.drop()

# create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

#train the chatbot
trainer.train(dialogs_sents)

List Trainer: [####################] 100%


### 3. Start a conversation with your chatbot, and discuss its strengths and weaknesses.

In [9]:
import random

In [10]:
print("Cornell: I will try to respond to you reasonably. If you want to exit, please type 'bye'.")

# below, the chatting takes place

while True: 
  
  user_input = input("User: ")
  user_input = user_input.lower()

  if(user_input!='bye'):
    if(user_input == 'thanks' or user_input =='thank you'):
      print("Cornell: You're welcome.")
      break
    else:
      if(greeting(user_input) != None):
        print("Cornell: " + greeting(user_input))
      else:
        print("Cornell: ", end = "")
        print(chatbot.get_response(user_input))
  else:
    print("Cornell: Bye! It was a great chat.")
    break

Cornell: I will try to respond to you reasonably. If you want to exit, please type 'bye'.
User: hey, what's up?
Cornell: I just wanted -- What?
User: Huh? Did something startle you?
Cornell: You promised you'd always take care of me.
User: No I didn't.
Cornell: The sub-surface structure shows that it was deliberately buried about four million years ago.
User: What was buried?
Cornell: You never wanted to go out with 'me, did you?
User: No
Cornell: Again.
User: Bye
Cornell: Bye! It was a great chat.
